In [1]:
import numpy as np
import matplotlib as plt
import scipy as scp
from generate_matrix import generate_symetric_matrix

# Methode des puissance itérée

## methode de la puissance

In [2]:
A_matrix = [generate_symetric_matrix(i**2) for i in range(2, 5)]

In [3]:
def methode_puissance(A : np.ndarray, y0 : np.ndarray=None, epsilon=10e-12, max_iter=10**4):
    n = A.shape[0]
    if y0 is None:
        y0 = np.random.randint(0, 9, (n))

    xk = y0/np.linalg.norm(y0)
    for k in range(max_iter):
        yk = A@xk
        xk = yk/np.linalg.norm(yk)
    return xk



def deflation(A : np.ndarray):
    n = A.shape[0]
    v = np.zeros((n, n))
    lambdas = np.ones(n)
    v0 = methode_puissance(A)
    v[:, 0] = v0
    lambdas[0] = (A@v0)@(v0)


    for i in range(1, n):
        A = A - (lambdas[i-1]/np.linalg.norm(v[:,i-1])**2)*np.outer(v[:,i-1], v[:,i-1])
        v[:, i] = methode_puissance(A, v[:, i-1])
        lambdas[i] = ((A@v[:, i])/v[:, i])[0]
    return lambdas, v


deflation(A_matrix[2])

(array([10. ,  9.4,  8.8,  8.2,  7.6,  7. ,  6.4,  5.8,  5.2,  4.6,  4. ,
         3.4,  2.8,  2.2,  1.6,  1. ]),
 array([[-0.0283998 ,  0.28417329,  0.17974181, -0.09075957, -0.2302834 ,
         -0.42184629, -0.22486979,  0.18730834,  0.19416639,  0.0866306 ,
          0.182331  ,  0.07142604, -0.06653332, -0.37856583,  0.3014877 ,
          0.48895161],
        [ 0.27375728,  0.42269505, -0.13263135,  0.49664879,  0.21999368,
          0.17775019, -0.07692673,  0.20243983, -0.07814589, -0.33959431,
          0.45037389, -0.09550569,  0.0665853 ,  0.0306652 ,  0.11900122,
         -0.04815079],
        [ 0.26678651,  0.14727695, -0.66225288, -0.34746841, -0.09328054,
         -0.15112971, -0.07573828,  0.05512508, -0.23812323,  0.24151778,
          0.17672762,  0.04029018, -0.01468998,  0.24314246, -0.24756679,
          0.19746381],
        [ 0.26879447,  0.3432847 ,  0.2814289 , -0.3123801 ,  0.39908959,
         -0.24807148,  0.41327136, -0.26464262, -0.20989037, -0.17321319,
   

## Methode QR 

In [10]:
def my_qr(A: np.ndarray):
    n = A.shape[0]
    Q = np.zeros((n, n))
    R = np.zeros((n, n))
    for k in range(n-1):
        R[k, k] =  np.linalg.norm(A[:, k])
        Q[:, k] = A[:, k]/R[k, k]

        R[k, k+1:] = Q.T[:, k]@A[:, k+1:]
        A[:, k+1:] = A[:, k+1:] - np.outer(Q[:, k],R[k, k+1:])

    R[n-1 ,n-1] = np.linalg.norm(A[:, n-1])
    Q[:, n-1] = A[:, n-1]/R[n -1, n - 1]
    return Q, R

def methode_qr(A : np.ndarray, qr_methode=my_qr, max_iter=10**4):
    n = A.shape[0]
    V = np.eye(n)
    Ak = A.copy()
    for k in range(max_iter):
        Qk, Rk = qr_methode(Ak)
        Ak = Rk@Qk
        V = V@Qk
    
    lambdas = np.diag(Ak)
    return lambdas, V

methode_qr(A_matrix[1], qr_methode=scp.linalg.qr)
    

(array([10.   ,  8.875,  7.75 ,  6.625,  5.5  ,  4.375,  3.25 ,  2.125,
         1.   ]),
 array([[ 0.42976912,  0.20737346, -0.27222584, -0.14249854,  0.08611994,
         -0.1091538 ,  0.69007907, -0.09893414,  0.41539587],
        [ 0.281687  ,  0.36149807, -0.63799326, -0.1481111 , -0.30545778,
         -0.17808089, -0.34748159,  0.04343456, -0.33667791],
        [-0.04253203,  0.44627239,  0.11860589, -0.57080125,  0.42033709,
          0.35758675, -0.26445076,  0.21164271,  0.19968017],
        [ 0.03070861,  0.25415892,  0.27669855,  0.02417521, -0.79447611,
          0.34817808, -0.01125698,  0.05498793,  0.31897184],
        [-0.07261677,  0.16348673, -0.2756699 ,  0.6289084 ,  0.19881652,
         -0.03329164, -0.30298255,  0.26080921,  0.54407997],
        [ 0.03051763,  0.27866973, -0.09208711,  0.33802561,  0.17611714,
          0.54909974,  0.03477639, -0.64577372, -0.21888271],
        [-0.58808166, -0.25164295, -0.45479473, -0.32362054, -0.14095861,
          0.03650155

In [9]:

# for n in range(len(A_matrix)):
A = A_matrix[0]
lambda_p, v_p = deflation(A)
lambda_qr, v_qr = methode_qr(A)
lambda_th, v_th = np.linalg.eigh(A)

erreurs_lambda = np.array([lambda_p, lambda_qr])
erreurs_v = np.array([v_p, v_qr])

erreurs_lambda = np.abs(np.subtract(erreurs_lambda , lambda_th))
erreurs_v = (np.subtract(erreurs_v, v_th))

erreurs_v = erreurs_v**2

print(erreurs_v)
print(erreurs_lambda)
erreurs_v.sum(axis=1)





    

    
    

[[[2.35147245e-01 1.41189133e+00 1.41189133e+00 3.04775071e-01]
  [1.20876098e+00 5.06876272e-01 5.06876272e-01 2.09883763e-01]
  [9.99911022e-05 4.02710470e-03 4.02710470e-03 3.97531445e-01]
  [5.55991782e-01 7.72052953e-02 7.72052953e-02 1.08780972e+00]]

 [[9.61461040e-01 1.67511127e-01 4.95435963e-01 1.12709726e-03]
  [1.32081755e+01 5.61276454e-01 4.81899216e-02 1.02818327e-01]
  [2.10697038e+01 4.51066993e-01 4.40706628e-01 9.62554945e-02]
  [1.55175871e+02 1.52508327e+01 1.56674867e-02 7.99799082e-01]]]
[[9. 3. 3. 9.]
 [9. 3. 7. 5.]]


array([[  2.        ,   2.        ,   2.        ,   2.        ],
       [190.41521102,  16.43068726,   1.        ,   1.        ]])